In [14]:
import os, re, sys
sys.path.append('D:\\liing_code\\spider_projects\\openlaw_spider\\middlewares')
import random
import requests
import pytesseract
from PIL import Image
from PIL import ImageOps
from aip import AipOcr
from collections import defaultdict
from settings import CAPTCHA_FILE, TIMEOUT

In [60]:
def do_verify():
    """
    验证码识别
    :return:
    """
    # 获取验证码图片
    # image_file = self.get_img()
    for image_file in os.listdir(CAPTCHA_FILE):
#         print(os.listdir(CAPTCHA_FILE))
        if 'captcha11.jpg' == image_file:
#             print(1)
            print(os.path.splitdrive(image_file)[-1].split('.')[0] + '>> ', end=' ', flush=True)
            # 验证码图片处理
            filename = CAPTCHA_FILE + image_file
            img = Image.open(filename)#.convert('L')
            print(pytesseract.image_to_string(img))
            img.show()
            img_invert = ImageOps.invert(img)
            image_name = filename.split('.')[0] + '.jpg'
            img_invert.save(image_name, 'jpeg')
            img_invert.show()
            # img = img_process(CAPTCHA_FILE + image_file)
            
            # 验证码图片识别
#             result = auto_captcha_identify(img)
#             print(result)
#             # 结果处理：去除非字母数字的
#             verify_code = []
#             for i in result:
#                 if i.isalpha():
#                     verify_code.append(i.upper())
#                 if i.isdigit():
#                     verify_code.append(i)
#             result = ''.join(verify_code)
#             print(result)
do_verify()

captcha11>>  W


In [37]:
def img_process(filename):
    """
    传入图片进行灰度、二值化、噪声处理
    :param filename: 图片文件
    :return: 处理后的图片对象
    """
    # 图片转化为灰度图
    image = Image.open(filename).convert('L')
    img = image
    img.show()
    img_invert = ImageOps.invert(img)
    image_name = filename.split('.')[0] + '.jpg'
    img_invert.save(image_name, 'jpeg')
    img_invert.show()
    return image_name

def img_threshold(image):
    """
    获取image文件像素值数量最多的像素点的像素值
    :param image: 验证码图片文件：流对象
    :return:
    """
    pixel_dict = defaultdict(int)    #　初始化一个像素点数量的字典,默认为int() —— 0
    w, h = image.size
    for i in range(w):
        for j in range(h):
            pixel = image.getpixel((i, j))
            pixel_dict[pixel] += 1
    max_pixel = max(pixel_dict.values())   # 像素值数量的最大值
    for k, v in pixel_dict.items():
        if v == max_pixel:
            threshold = k
            return threshold
        else:
            continue

def img_bin_process(threshold):
    """
    根据阈值获取灰度图转为二值图的映射列表
    :param threshold: 灰度图像素阈值
    :return:
    """
    table = []
    for i in range(256):
        rate = 0.6  # 在threshold的适当范围内进行二值转化
        if threshold * (1 - rate) <= i <= threshold * (1 + rate):
            table.append(1)    # 在阈值附近的去除，其他保留
        else:
            table.append(0)
    return table

def img_rid_noise(image):
    """
    去掉二值处理后图片的噪声点
    :param image: 二值处理后的图片流对象
    :return: 去除噪声后的图片
    """
    w, h = image.size
    noises = []    # 记录噪声点位置
    pixes = []
    for i in range(1, w - 1):
        for j in range(1, h - 1):   # 遍历图片中的点（去除边缘）
            pixel_b_counts = []     # 记录该点附近黑色像素的数量
            count = 0
            for m in range(i-1, i+2):
                for n in range(j-1, j+2):   # 取该点为中心的九宫格遍历
                    pix = image.getpixel((m, n))
                    if pix != 1:    # 1为白色，0为黑色
                        count += 1
                        # pixel_b_counts.append(pix)
            if count <= 2:          # 该像素附近的黑色像素数量 <= 4， 则该像素为噪声
                noises.append((i, j))
            if count >= 8:          # 该像素附近的黑色像素数量 >= 8, 则该像素不是噪声
                pixes.append((i, j))
    for noise in noises:            # 噪声点像素设为1  白色， 去除噪声
        image.putpixel(noise, 1)
    for pix in pixes:
        image.putpixel(pix, 0)      # 非噪声点像素设为0  黑色， 修复图像
    
    plt.imshow(image)
    plt.show()
    return image

def auto_captcha_identify(img):
    """
    自动验证码识别
    :return:
    """
    result = pytesseract.image_to_string(img)
    print(result)
    return result

# 自定义算法
def do_verify():
    """
    验证码识别
    :return:
    """
    # 获取验证码图片
    # image_file = self.get_img()
    for image_file in os.listdir(CAPTCHA_FILE):
        if 'captcha1.jpg' in image_file:
            print(os.path.splitdrive(image_file)[-1].split('.')[0] + '>> ', end=' ', flush=True)
            # 验证码图片处理
            img = img_process(CAPTCHA_FILE + image_file)
            # 验证码图片识别
            result = auto_captcha_identify(img)
            print(result)
            # 结果处理：去除非字母数字的
            verify_code = []
            for i in result:
                if i.isalpha():
                    verify_code.append(i.upper())
                if i.isdigit():
                    verify_code.append(i)
            result = ''.join(verify_code)
            print(result)
do_verify()

# plt.imshow(img)
# plt.show()